In [ ]:
import os
import json
import time
from cryptography.fernet import Fernet

# Generate encryption keys for each component
key_database = {
    "client": Fernet.generate_key(),
    "AS": Fernet.generate_key(),
    "TGS": Fernet.generate_key(),
    "service": Fernet.generate_key()
}

# ==========================
# 🔐 Encryption & Decryption
# ==========================

def encrypt_data(key, data):
    """Encrypts a JSON object using Fernet symmetric encryption."""
    return Fernet(key).encrypt(bytes(json.dumps(data), "utf-8")) 

def decrypt_data(key, encrypted_data):
    """Decrypts a JSON object using Fernet symmetric encryption."""
    return json.loads(Fernet(key).decrypt(encrypted_data))


# ==========================
# 🎟️ Authentication Server (AS) - Issues Ticket Granting Ticket (TGT)
# ==========================

def authentication_server(client_id):
    """Simulates the Authentication Server issuing a TGT to the client."""
    session_key = Fernet.generate_key()

    return encrypt_data(key_database["AS"], {
        "client_id" : client_id,
        "session_key" : bytes.decode(session_key),
        "nonce" : generate_nonce(),
        "issue_time" : time.time()
    }), session_key

# ==========================
# 🎫 Ticket Granting Server (TGS) - Issues Service Ticket
# ==========================

def ticket_granting_server(encrypted_tgt, client_session_key):
    """Simulates the Ticket Granting Server issuing a service ticket."""
    try:
        tgt = decrypt_data(key_database["AS"], encrypted_tgt)

        if time.time() - tgt["issue_time"] > 300: return "TGT Expired", None
        if tgt["session_key"] != bytes.decode(client_session_key): return "TGT session key invaild", None

        service_session_key = Fernet.generate_key()

        service_ticket = {
            "client_id": tgt["client_id"],
            "service_session_key": bytes.decode(service_session_key),
            "issued_at": time.time(),
            "nonce": generate_nonce()
        }

        # TODO: Encrypt service ticket with TGS key before returning
        encrypted_service_ticket = encrypt_data(key_database["TGS"], service_ticket)

        return encrypted_service_ticket, service_session_key

    except Exception as e:
        return f"❌ Error in TGS: {str(e)}", None

# ==========================
# 🏛️ Service Server - Grants Access
# ==========================

def service_server(encrypted_service_ticket):
    """Validates and decrypts the service ticket before granting access."""
    try:
        ticket = decrypt_data(key_database["TGS"], encrypted_service_ticket)

        if time.time() - ticket["issued_at"] > 300: return "Service Ticket Expired", None

        return f"✅ Access granted to {ticket["client_id"]} with key {ticket["service_session_key"]}"

    except Exception as e:
        return f"❌ Service Authentication Failed: {str(e)}"
    
def generate_nonce(length=8):
    """Generates a random string of hex values of any given length"""
    return bytes.hex(os.urandom(length))

# ==========================
# 🖥️ Interactive CLI
# ==========================

def main():
    # """Interactive command-line interface for running Kerberos authentication steps."""
    client_id = "client"
    tgt = None
    client_session_key = None
    service_ticket = None
    service_session_key = None

    while True:
        print("\n===== Kerberos Authentication System =====")
        print("1. Request Ticket Granting Ticket (TGT)")
        print("2. Request Service Ticket")
        print("3. Access Protected Service")
        print("4. Exit")
        choice = input("Enter your choice: ").strip()

        if choice == "1":
            tgt, client_session_key = authentication_server(client_id)

            service_ticket = None
            service_session_key = None

            print(f"\n🔐 Received TGT:")
            print(tgt)
            print(f"\n🔐 Received Client Session Key:")
            print(client_session_key)


        elif choice == "2":
            service_ticket, service_session_key = ticket_granting_server(tgt, client_session_key)
            print(f"\n🎟️ Received Service Ticket:")
            print(service_ticket)
            print(f"\n🎟️ Received Service Session Key:")
            print(service_session_key)

        elif choice == "3":
            print(f"\n {service_server(service_ticket)}")

        elif choice == "4":
            print("\n🚪 Exiting the system.")
            break   

        else:
            print("\n❌ Invalid choice, please enter a valid option.")

    # Encrypt and Deecrypt data test
    # test_key = key_database["client"] 
    # test_data = {
    #     "test" : "Hello World",
    #     "foo" : "bar",
    #     "123" : 456
    # }

    # print(f"Input: {test_data}")
    # encrypted = encrypt_data(test_key, test_data)
    # print(f"Encrypted token: {encrypted}")
    # print(f"Decrypted data: {decrypt_data(test_key, encrypted)}")\
    
    # Authentication Server data test
    # tgt, client_sess = authentication_server("client")
    # print("Encrypted TGT:", tgt)
    # print("Client session key:", client_sess)
    # print("Decrypted TGT :", decrypt_data(key_database["AS"], tgt))

    # Ticket Granting Server data test
    # tgt, client_sk = authentication_server("client")
    # svc_ticket, svc_sk = ticket_granting_server(tgt, client_sk)

    # print("Encrypted service ticket:", svc_ticket)
    # print("Decrypted service ticket (debug):", decrypt_data(key_database["TGS"], svc_ticket))
    # print("Service session key:", svc_sk)

    #Service Server
    # tgt, client_sk = authentication_server("client")
    # svc_ticket = ticket_granting_server(tgt, client_sk)[0]
    # print(service_server(svc_ticket))


# Run the interactive simulation
if __name__ == "__main__":
    main()
